# Anysee Tutorial - Face Detection / Recognition basic API usages

In this example we will first show how to use the Face Detection API to detect faces of images on the internet. 

In [1]:
import base64
import logging
import requests
from requests.structures import CaseInsensitiveDict

region = 'ap-northeast-1' # this is the region we picked when we created the instance
endpoint = 'api/v1/entities/faces/detect'

url = f'https://anysee.{region}.cloud.japancv.co.jp/{endpoint}'
apikey = 'e6a26ecd-7b43-4fc8-bfad-04e82e0a3848'
headers = CaseInsensitiveDict()
headers['Content-Type'] = 'application/json'
headers['api-key'] = apikey

Here we have the REST request body for doing the detection. See the [docs](https://docs.cloud.japancv.co.jp/docs/anysee-introduction) for more information.

For the parameters, img is the image in base64 format.

`pos` stands for the position, `ang` stands for angle, `lnd` stands for landmark locations, `qua` stands for the quality of the face, and `att` stands for the attributes.

In [2]:
def sendRequest(img:str, pos=False, ang=False, lnd=False, qua=False, att=False):
    data = {}
    image = {}
    rd = {}
    rd["position"] = pos
    rd["angle"] = ang
    rd["landmarks"] = lnd
    rd["quality"] = qua
    rd["attributes"] = att
    image['data'] = img
    image['autoRotate'] = True
    image['returnDetails'] = rd
    data['model'] = 'JCV_FACE_K25000'
    data['image'] = image
    success = True
    try:
        res = requests.post(url, json=data, headers=headers, timeout=10)
    except Exception as e:
        logging.warning(f'Anysee timeout! {e}')
        res = 'Anysee endpoint timeout'
        success = False
    if res == 'Anysee endpoint timeout' or res.status_code != 200:
        success = False
    return res, success

The API requires a base64 format of the image data, so let's make a function to convert the data to base64 format.

In [3]:
def b64(image_path):
    with open(image_path, "rb") as image_file:
        encode = base64.b64encode(image_file.read()).decode('utf-8')
    return encode

We are going to download a random image from the internet for the demo. Feel free to change the image used here. We'll read it in again later.

In [4]:
img_data = requests.get('https://img.kyodonews.net/english/public/images/posts/ed58f81ded6f4ac7f66e61fed97fbe39/photo_l.jpg').content
with open('sample.jpg', 'wb') as handler:
    handler.write(img_data)

Now that we have the example image that we need, let's start doing some face detection!
Read the imageand send the request. 

In [5]:
req_img = b64('sample.jpg')
res, suc = sendRequest(req_img, pos=True, ang=True, lnd=True, qua=True, att=True)

In [6]:
type(res)
res.json()

{'model': 'JCV_FACE_K25000',
 'count': 1,
 'entities': [{'rotated': 0,
   'details': {'position': {'top': 364,
     'left': 290,
     'width': 377,
     'height': 394},
    'angle': {'yaw': -0.4502186,
     'pitch': 2.913058,
     'roll': -0.5153649,
     'centerX': 478,
     'centerY': 561},
    'landmarks': [{'x': 297, 'y': 456},
     {'x': 297, 'y': 482},
     {'x': 298, 'y': 508},
     {'x': 300, 'y': 535},
     {'x': 303, 'y': 560},
     {'x': 307, 'y': 586},
     {'x': 311, 'y': 612},
     {'x': 318, 'y': 636},
     {'x': 328, 'y': 660},
     {'x': 342, 'y': 682},
     {'x': 358, 'y': 703},
     {'x': 377, 'y': 719},
     {'x': 396, 'y': 736},
     {'x': 418, 'y': 750},
     {'x': 439, 'y': 762},
     {'x': 463, 'y': 767},
     {'x': 487, 'y': 767},
     {'x': 513, 'y': 766},
     {'x': 537, 'y': 760},
     {'x': 558, 'y': 748},
     {'x': 579, 'y': 734},
     {'x': 597, 'y': 717},
     {'x': 615, 'y': 697},
     {'x': 630, 'y': 677},
     {'x': 642, 'y': 655},
     {'x': 652, 'y